<a href="https://colab.research.google.com/github/AdamFulton/COMP3100StageTwo/blob/main/EEG_Emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [319]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,SubsetRandomSampler
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle as pickle
from sklearn.preprocessing import StandardScaler

In [320]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat May 14 05:51:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    33W / 250W |   8707MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [321]:
from google.colab import drive

drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [322]:
def read_eeg_signal_from_file(filename):
    x = pickle._Unpickler(open(filename, 'rb'))
    x.encoding = 'latin1'
    p = x.load()
    return p



In [323]:
files = []
for n in range(1, 33): 
    s = ''
    if n < 10:
        s += '0'
    s += str(n)
    files.append(s)
print(files)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']


In [324]:

labels = []
data = []

for i in files: 
    filename = "/content/gdrive/MyDrive/data_preprocessed_python/s" + i + ".dat"
    trial = read_eeg_signal_from_file(filename)
    labels.append(trial['labels'])
    data.append(trial['data'])

labels = np.array(labels)
labels = labels.flatten()
labels = labels.reshape(1280, 4)

data = np.array(data)
data = data.flatten()
data = data.reshape(1280, 40, 8064)

In [325]:
print("Labels: ", labels.shape)
print("Data: ", data.shape)

Labels:  (1280, 4)
Data:  (1280, 40, 8064)


In [326]:
df_label_ratings = pd.DataFrame({'Valence': labels[:,0], 'Arousal': labels[:,1]})
print(df_label_ratings.describe())

           Valence      Arousal
count  1280.000000  1280.000000
mean      5.254313     5.156711
std       2.130816     2.020499
min       1.000000     1.000000
25%       3.867500     3.762500
50%       5.040000     5.230000
75%       7.050000     6.950000
max       9.000000     9.000000


In [329]:
print(df_label_ratings.head(15))

    Valence  Arousal
0      7.71     7.60
1      8.10     7.31
2      8.58     7.54
3      4.94     6.01
4      6.96     3.92
5      8.27     3.92
6      7.44     3.73
7      7.32     2.55
8      4.04     3.29
9      1.99     4.86
10     2.99     2.36
11     2.71     2.77
12     1.95     3.12
13     4.18     2.24
14     3.17     8.08


In [330]:

df_hahv = df_label_ratings[(df_label_ratings['Valence'] >= np.median(labels[:,0])) & (df_label_ratings['Arousal'] >= np.median(labels[:,1]))]

df_lahv = df_label_ratings[(df_label_ratings['Valence'] >= np.median(labels[:,0])) & (df_label_ratings['Arousal'] < np.median(labels[:,1]))]

df_halv = df_label_ratings[(df_label_ratings['Valence'] < np.median(labels[:,0])) & (df_label_ratings['Arousal'] >= np.median(labels[:,1]))]

df_lalv = df_label_ratings[(df_label_ratings['Valence'] < np.median(labels[:,0])) & (df_label_ratings['Arousal'] < np.median(labels[:,1]))]

In [331]:
print("Positive Valence:", str(len(df_hahv) + len(df_lahv)))
print("Negative Valence:", str(len(df_halv) + len(df_lalv)))
print("High Arousal:", str(len(df_hahv) + len(df_halv)))
print("Low Arousal:", str(len(df_lahv) + len(df_lalv)))

Positive Valence: 680
Negative Valence: 600
High Arousal: 640
Low Arousal: 640


In [ ]:
def positive_valence(trial):
    return 1 if labels[trial,0] >= np.median(labels[:,0]) else 0 

def high_arousal(trial):
    return 1 if labels[trial,1] >= np.median(labels[:,1]) else 0

In [332]:
labels_encoded = []
for i in range (len(labels)):
  labels_encoded.append([positive_valence(i), high_arousal(i)])

labels_encoded = np.reshape(labels_encoded, (1280, 2))
df_labels = pd.DataFrame(data=labels_encoded, columns=["Positive Valence", "High Arousal",])



In [333]:
df_valence = df_labels['Positive Valence']

df_arousal = df_labels['High Arousal']

df_arousal = df_arousal.to_numpy()

df_valence = df_valence.to_numpy()




In [334]:
eeg_channels = np.array(["Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1", "P3", "P7", "PO3", "O1", "Oz", "Pz", "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8", "PO4", "O2"])
peripheral_channels = np.array(["hEOG", "vEOG", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])


In [335]:
eeg_data = []
for i in range (len(data)):
    for j in range (len(eeg_channels)):
        eeg_data.append(data[i,j])
eeg_data = np.reshape(eeg_data, (len(data), len(eeg_channels), len(data[0,0])))
print(eeg_data.shape)

(1280, 32, 8064)


In [336]:
train_size = int(0.8 * len(eeg_data))
valid_size = len(eeg_data) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(eeg_data, [train_size, valid_size])
train_dataset = np.array(train_dataset)
valid_dataset = np.array(valid_dataset)
scaler = StandardScaler()
X_train = scaler.fit_transform(train_dataset.reshape(-1, train_dataset.shape[-1])).reshape(train_dataset.shape)
X_test = scaler.transform(valid_dataset.reshape(-1, valid_dataset.shape[-1])).reshape(valid_dataset.shape)



train = []
valid = []
print(len(X_test))
for i in range(valid_size,train_size):

    valid.append((X_train[i], df_valence[i]))
  

for i in range(0,len(X_test)):

    train.append((X_test[i],df_valence[i]))




print(train[0])

256
(array([[-1.26700192, -0.69311858, -0.46014445, ..., -0.22123851,
        -0.47546754, -0.28220913],
       [-1.80193374, -0.84759446, -0.13018708, ...,  0.08564208,
        -0.51966667, -0.44539885],
       [-1.25159991, -0.34756283,  0.7239788 , ...,  0.07629579,
        -0.63620148, -0.55901563],
       ...,
       [-1.62262069, -0.57195197,  0.85083543, ..., -0.00479637,
        -1.01674883, -0.71999102],
       [-1.69913995, -0.63997153,  0.76780417, ...,  0.0590668 ,
        -1.11963866, -0.87837184],
       [-2.12438508, -0.78088265,  1.01784947, ..., -0.00246079,
        -1.24424848, -0.97398621]]), 1)


In [337]:
batch_size=16


train_dl = DataLoader(train,batch_size)


valid_dl = DataLoader(valid,batch_size)


len(train)

256

In [338]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv1d(inp, oup, 2, stride, 1, bias=False),
                nn.BatchNorm1d(oup),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.25),
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv1d(inp, inp, 2, stride, 1, groups=inp, bias=False),
                nn.BatchNorm1d(inp),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.25),
                
              
    
                nn.Conv1d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm1d(oup),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.25),
               
               
            )

        self.model = nn.Sequential(
            conv_bn( 32,  128, 2), 
            conv_dw( 128,  200, 1),
            conv_dw( 200, 225, 2),
            conv_dw(225, 245, 1),
            conv_dw(245, 256, 2),
            conv_dw(256, 412, 1),
            conv_dw(412, 512, 2),
            conv_dw(512, 512, 1),
            nn.AvgPool1d(260),
        )

        self.classifier = nn.Sequential(
        nn.Linear(512, 512),
      	nn.ReLU(),
        nn.Dropout(p=0.75),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(p=0.75),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Dropout(p=0.75),
        nn.Linear(128, 64),
        nn.Softmax(),

        nn.Linear(64,2)
        

        )
    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 512)
        x = self.classifier(x)
        return x

In [339]:
def get_default_device():

  if torch.cuda.is_available():
    return torch.device("cuda")

  else:
    return torch.device("cpu")

In [340]:
device = get_default_device();
device

device(type='cuda')

In [341]:
def to_device(data,device):

  if isinstance(data,(list,tuple)):
    return [to_device(x,device) for x in data]

  return data.to(device,non_blocking=True)

In [342]:
for data,labels in train_dl:
  print(data.shape)
  data = to_device(data,device)
  print(data.device)
  break

torch.Size([16, 32, 8064])
cuda:0


In [343]:
class DeviceDataLoader():
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
       
        return len(self.dl)

In [344]:
train_dl  = DeviceDataLoader(train_dl,device)
valid_dl = DeviceDataLoader(valid_dl,device)
model = Net()
to_device(model,device)

Net(
  (model): Sequential(
    (0): Sequential(
      (0): Conv1d(32, 128, kernel_size=(2,), stride=(2,), padding=(1,), bias=False)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (1): Sequential(
      (0): Conv1d(128, 128, kernel_size=(2,), stride=(1,), padding=(1,), groups=128, bias=False)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Dropout(p=0.25, inplace=False)
      (4): Conv1d(128, 200, kernel_size=(1,), stride=(1,), bias=False)
      (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): Dropout(p=0.25, inplace=False)
    )
    (2): Sequential(
      (0): Conv1d(200, 200, kernel_size=(2,), stride=(2,), padding=(1,), groups=200, bias=False)
      (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, aff

In [345]:
def accuracy(outputs,labels):
    _, preds = torch.max(outputs,dim=1)
    return torch.sum(preds == labels).item() / len(preds)

In [ ]:
model = Net()
val_loss, _,val_acc =evaluate(model,nn.CrossEntropyLoss(),valid_dl,metric=accuracy)
print('Loss: {:.4f}, Accuracy: {:.4f}'.format(val_loss,val_acc))

In [346]:

model = Net().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
min_valid_loss = np.inf

In [347]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for e in range(100):
    train_loss = 0.0
    model.train()     
    for data, labels in train_dl:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        data = data.float()
        target = model(data)
        loss = loss_function(target,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    valid_loss = 0.0
    model.eval()     
    for data, labels in valid_dl:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        data = data.float()
        target = model(data)
        loss = loss_function(target,labels)
        valid_loss = loss.item() * data.size(0)

    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_dl)} \t\t Validation Loss: {valid_loss / len(valid_dl)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_model.pth')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 1 		 Training Loss: 0.6894524171948433 		 Validation Loss: 0.22324277957280478
Validation Loss Decreased(inf--->10.715653) 	 Saving The Model
Epoch 2 		 Training Loss: 0.6893130801618099 		 Validation Loss: 0.22306295235951742
Validation Loss Decreased(10.715653--->10.707022) 	 Saving The Model
Epoch 3 		 Training Loss: 0.6892875544726849 		 Validation Loss: 0.22287797927856445
Validation Loss Decreased(10.707022--->10.698143) 	 Saving The Model
Epoch 4 		 Training Loss: 0.6892017722129822 		 Validation Loss: 0.22268142302831015
Validation Loss Decreased(10.698143--->10.688708) 	 Saving The Model
Epoch 5 		 Training Loss: 0.689043965190649 		 Validation Loss: 0.22248599926630655
Validation Loss Decreased(10.688708--->10.679328) 	 Saving The Model
Epoch 6 		 Training Loss: 0.6888999566435814 		 Validation Loss: 0.2222959597905477
Validation Loss Decreased(10.679328--->10.670206) 	 Saving The Model
Epoch 7 		 Training Loss: 0.6889811083674431 		 Validation Loss: 0.22210816542307535